In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

from datetime import datetime, time
from google.cloud import bigquery
from google.oauth2 import service_account

from matplotlib import pyplot as plt

%matplotlib inline

pd.set_option('display.max_columns', None)
# pd.reset_option(‘max_columns’)

In [2]:
X=pd.read_csv('x.csv')
Y=pd.read_csv('y.csv')

In [3]:
display(X.head())
display(Y.head())

,mkt_datetime,int_nuc_sch,int_gas_sch,int_hyd_sch,int_win_sch,imp_mic_sch,imp_new_sch,imp_que_sch,ont_avg_dem,exp_mic_sch,exp_new_sch,exp_que_sch
0,2020-01-01T00:00,11554,216,2857,2113,0,0,9,13305,-800,-1285,-1273
1,2020-01-01T01:00,11554,216,2834,1595,0,0,9,12870,-800,-1285,-1273
2,2020-01-01T02:00,11554,216,2834,1506,0,0,9,12512,-800,-1390,-1273
3,2020-01-01T03:00,11554,216,2835,1592,0,0,9,12370,-800,-1443,-1273
4,2020-01-01T04:00,11554,216,2911,1479,0,0,9,12469,-800,-1440,-1273


,mkt_datetime,ont_ene
0,2020-01-01T00:00,0.0
1,2020-01-01T01:00,0.0
2,2020-01-01T02:00,0.0
3,2020-01-01T03:00,0.0
4,2020-01-01T04:00,0.0


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23568 entries, 0 to 23567
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   mkt_datetime  23568 non-null  object
 1   int_nuc_sch   23568 non-null  int64 
 2   int_gas_sch   23568 non-null  int64 
 3   int_hyd_sch   23568 non-null  int64 
 4   int_win_sch   23568 non-null  int64 
 5   imp_mic_sch   23568 non-null  int64 
 6   imp_new_sch   23568 non-null  int64 
 7   imp_que_sch   23568 non-null  int64 
 8   ont_avg_dem   23568 non-null  int64 
 9   exp_mic_sch   23568 non-null  int64 
 10  exp_new_sch   23568 non-null  int64 
 11  exp_que_sch   23568 non-null  int64 
dtypes: int64(11), object(1)
memory usage: 2.2+ MB


In [5]:
print('>> X')
display(X.nunique())
print('>> Y')
display(Y.nunique())

>> X


mkt_datetime    23565
int_nuc_sch      1068
int_gas_sch      4982
int_hyd_sch      3310
int_win_sch      4090
imp_mic_sch       465
imp_new_sch       385
imp_que_sch      2046
ont_avg_dem      8877
exp_mic_sch      1102
exp_new_sch      1634
exp_que_sch      1475
dtype: int64

>> Y


mkt_datetime    23554
ont_ene          7147
dtype: int64

In [6]:
X[X['mkt_datetime'].duplicated() == True]

,mkt_datetime,int_nuc_sch,int_gas_sch,int_hyd_sch,int_win_sch,imp_mic_sch,imp_new_sch,imp_que_sch,ont_avg_dem,exp_mic_sch,exp_new_sch,exp_que_sch
1611,2020-03-08T03:00,10020,178,3189,2842,0,0,9,12989,-1450,-1407,-432
10515,2021-03-14T03:00,8345,216,2768,2830,0,0,9,12793,-1000,-517,-66
19251,2022-03-13T03:00,8345,827,3461,2274,0,0,1549,14898,-1450,-168,-214


In [7]:
# X.iloc[1608:1613]
# X.iloc[10513:10517]
# X.iloc[19249:19254]

In [8]:
X.at[1610,'mkt_datetime']='2020-03-08T02:00'
X.at[10514,'mkt_datetime']='2021-03-14T02:00'
X.at[19250,'mkt_datetime']='2022-03-13T02:00'

In [9]:
print('>> X ::', min(X.mkt_datetime), max(X.mkt_datetime))
print('>> Y ::', min(Y.mkt_datetime), max(Y.mkt_datetime))

>> X :: 2020-01-01T00:00 2022-09-08T23:00
>> Y :: 2020-01-01T00:00 2022-09-08T09:00


### Merging both dataframes X and Y

In [10]:
X['mkt_datetime'] = pd.to_datetime(X.mkt_datetime)
Y['mkt_datetime'] = pd.to_datetime(Y.mkt_datetime)

In [11]:
merged_data = pd.merge(X,Y,how='left',on = 'mkt_datetime')

In [13]:
merged_data=merged_data.set_index('mkt_datetime')

In [14]:
merged_data

,int_nuc_sch,int_gas_sch,int_hyd_sch,int_win_sch,imp_mic_sch,imp_new_sch,imp_que_sch,ont_avg_dem,exp_mic_sch,exp_new_sch,exp_que_sch,ont_ene
mkt_datetime,,,,,,,,,,,,
2020-01-01 00:00:00,11554,216,2857,2113,0,0,9,13305,-800,-1285,-1273,0.0
2020-01-01 01:00:00,11554,216,2834,1595,0,0,9,12870,-800,-1285,-1273,0.0
2020-01-01 02:00:00,11554,216,2834,1506,0,0,9,12512,-800,-1390,-1273,0.0
2020-01-01 03:00:00,11554,216,2835,1592,0,0,9,12370,-800,-1443,-1273,0.0
2020-01-01 04:00:00,11554,216,2911,1479,0,0,9,12469,-800,-1440,-1273,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-08 19:00:00,9415,4183,4491,339,366,650,9,18660,-355,-380,-9,NaN
2022-09-08 20:00:00,9415,3976,4218,406,305,250,9,17742,-418,-380,-9,NaN
2022-09-08 21:00:00,9415,3177,3948,484,280,0,9,16484,-645,-621,-9,NaN


In [15]:
merged_data.to_csv('merged_data.csv')